# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata

---
## Confirmed cases

In [ ]:
import os

import numpy as np
import pandas as pd
import pymc3 as pm

from covidvu import utils
from covidvu import vujson
import numpy as np
import covidvu.visualize as vis

pd.options.mode.chained_assignment = None


In [ ]:
N_SAMPLES = 1000
N_DAYS_PREDICT = 14

In [ ]:
confirmedCases, _, _, _ = vujson._main('confirmed')

In [ ]:
casesData = confirmedCases['United Kingdom']
casesData = casesData.iloc[-20:]
casesLogData = np.log(casesData + 1).values
t = np.arange(casesData.shape[0])

In [ ]:
fig, ax = vis.plot()
ax.plot(t, casesData)

In [ ]:
fig, ax = vis.plot()
ax.plot(t, casesLogData)

In [ ]:
logRegModel = pm.Model()
with logRegModel:
    # Priors
    logCarryingCapacity = pm.Uniform('logCarryingCapacity', 3,10)
    midPoint            = pm.Uniform('midPoint', 0, 1e3)
    growthRate          = pm.Uniform('growthRate', 0, 1)
    sigma               = pm.Uniform('sigma', 0, 10)
    #sigma               = pm.Uniform('sigma', 0, 1e5)
    
    carryingCap = 10**logCarryingCapacity
    casesLin = carryingCap/(1 + np.exp(-1.0 * growthRate * (t - midPoint)))
    casesLog = np.log(casesLin + 1)
    
    
    casesObsLog = pm.Normal('casesObsLog', mu=casesLog, sigma=sigma, observed=casesLogData)
    

In [ ]:
with logRegModel:
    traceLog = pm.sample(N_SAMPLES, chains=1)

In [ ]:
pm.plots.traceplot(traceLog);

In [ ]:
tStartPredict = t[-1]+1
tPredict = np.arange(tStartPredict, tStartPredict + N_DAYS_PREDICT)

predictions = np.zeros((N_DAYS_PREDICT, N_SAMPLES))

for i in range(N_SAMPLES):
    carryingCap = 10**traceLog['logCarryingCapacity'][i]

    predictions[:, i] = carryingCap/(1 + np.exp(-1.0 * traceLog['growthRate'][i] * (tPredict - traceLog['midPoint'][i])))
    

In [ ]:
predictionsLow  = np.percentile(predictions, 2.5, axis=1)
predictionsHigh = np.percentile(predictions, 100 - 2.5, axis=1)
predictionsMean = predictions.mean(axis=1)

In [ ]:
fig, ax = vis.plot()
ax.plot(t, casesData, '-k', label = 'Observed')
ax.plot(tPredict, predictionsMean, '-.k', label = 'Posterior mean')
ax.fill_between(tPredict,
                predictionsLow,
                predictionsHigh,
                color='red',
                alpha=0.5, 
                label = r"95\% BCI");

In [ ]:
with logRegModel:
    traceTest = pm.sample(tune=100, draws=100 , chains=1)